In [1]:
!pip install qiskit qiskit-aer --upgrade

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.0/8.0 MB 63.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.4/12.4 MB 69.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 44.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.5/49.5 kB 2.0 MB/s eta 0:00:00


In [2]:
from qiskit import QuantumCircuit, transpile
from qiskit_aer import AerSimulator

# Initialize simulator
sim = AerSimulator()

# ============================================================
# 1) Control/Target Variation
# ============================================================
print("\n--- 1) Control/Target Variation ---")
qc1 = QuantumCircuit(2, 2)

qc1.h(0)
qc1.cx(1, 0)   # swapped control and target
qc1.cz(1, 0)   # swapped control and target
qc1.swap(0, 1)
qc1.measure_all()

compiled1 = transpile(qc1, sim)
result1 = sim.run(compiled1, shots=1024).result()
counts1 = result1.get_counts()

print(qc1.draw())
print("Measurement Counts:", counts1)
print("Explanation: Changing control/target qubits changes how entanglement forms. "
      "In CNOT and CZ, control determines when the operation happens. "
      "Swapping them changes the correlation between qubits.")

# ============================================================
# 2) Gate Removal (Remove SWAP)
# ============================================================
print("\n--- 2) Removing the SWAP gate ---")
qc2 = QuantumCircuit(2, 2)

qc2.h(0)
qc2.cx(0, 1)
qc2.cz(0, 1)
# qc2.swap(0, 1)  # removed
qc2.measure_all()

compiled2 = transpile(qc2, sim)
result2 = sim.run(compiled2, shots=1024).result()
counts2 = result2.get_counts()

print(qc2.draw())
print("Measurement Counts:", counts2)
print("Explanation: Removing SWAP keeps qubits in their original order. "
      "The SWAP gate exchanges qubit states, so without it, results are recorded "
      "in their initial register positions, changing which qubit’s value appears in which bit.")

# ============================================================
# 3) Add Another Hadamard (on second qubit before CNOT)
# ============================================================
print("\n--- 3) Adding Hadamard to second qubit before CNOT ---")
qc3 = QuantumCircuit(2, 2)

qc3.h(0)
qc3.h(1)        # new Hadamard on qubit 1
qc3.cx(0, 1)
qc3.cz(0, 1)
qc3.swap(0, 1)
qc3.measure_all()

compiled3 = transpile(qc3, sim)
result3 = sim.run(compiled3, shots=1024).result()
counts3 = result3.get_counts()

print(qc3.draw())
print("Measurement Counts:", counts3)
print("Explanation: Adding H to the second qubit puts it into superposition before entanglement. "
      "This increases the range of possible measurement outcomes because both qubits can now interfere.")




--- 1) Control/Target Variation ---
        ┌───┐┌───┐       ░ ┌─┐   
   q_0: ┤ H ├┤ X ├─■──X──░─┤M├───
        └───┘└─┬─┘ │  │  ░ └╥┘┌─┐
   q_1: ───────■───■──X──░──╫─┤M├
                         ░  ║ └╥┘
   c: 2/════════════════════╬══╬═
                            ║  ║ 
meas: 2/════════════════════╩══╩═
                            0  1 
Measurement Counts: {'10 00': 515, '00 00': 509}
Explanation: Changing control/target qubits changes how entanglement forms. In CNOT and CZ, control determines when the operation happens. Swapping them changes the correlation between qubits.

--- 2) Removing the SWAP gate ---
        ┌───┐         ░ ┌─┐   
   q_0: ┤ H ├──■───■──░─┤M├───
        └───┘┌─┴─┐ │  ░ └╥┘┌─┐
   q_1: ─────┤ X ├─■──░──╫─┤M├
             └───┘    ░  ║ └╥┘
   c: 2/═════════════════╬══╬═
                         ║  ║ 
meas: 2/═════════════════╩══╩═
                         0  1 
Measurement Counts: {'00 00': 538, '11 00': 486}
Explanation: Removing SWAP keeps qubits in their ori